In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from flask import Flask, request, render_template, g 
from bs4 import BeautifulSoup
import urllib.request
from flask_ngrok import run_with_ngrok
from flask_ngrok import *
from urllib.request import Request, urlopen
from datetime import datetime
import pandas as pd
import csv
import sqlite3
import json 
import re

In [6]:
from collections import Counter 
from string import punctuation
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS as stop_words
import spacy 
from spacy.lang.en.stop_words import STOP_WORDS
import nltk 
from nltk.corpus import stopwords

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_extraction.stop_words module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_extraction.text. Anything that cannot be imported from sklearn.feature_extraction.text is now part of the private API.
  warnings.warn(message, FutureWarning)


In [7]:
nltk.download('stopwords')
swa = stopwords.words('arabic')
swf = set(stopwords.words('french'))
sws = set(stopwords.words('spanish'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
#tokenizing sentences into words
def tokenizer(s):
    tokens = []
    for word in s.split(' '):
        tokens.append(word.strip().lower()) #strip remove spaces at the beginning and at the end of the string
    return tokens
#tokenizing document into sentences 

def sent_tokenizer(s):
    sents = []
    for sent in s.split('.'):
        sents.append(sent.strip())
    return sents

In [9]:
!pip install flask_ngrok

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
cd /content/drive/My Drive/quick


/content/drive/My Drive/quick


In [12]:
def get_content(url):
  req_obj = requests.get(url)
  text = req_obj.text
  soup = BeautifulSoup(text)
  title = soup.find("div" , {"class" : "col article-container"})
  content_a = soup.findAll("div", {"class": "content"})
  content = ''
  for x in content_a:
    for c in x.findAll('p'):
      content += c.text + "\n"
  return content

In [13]:
def count_words(tokens):
    word_counts = {}
    for token in tokens:
        if token not in stop_words and token not in STOP_WORDS and token not in swa and token not in sws and token not in swf and token not in punctuation:
            if token not in word_counts.keys():
                word_counts[token] = 1
            else:
                word_counts[token] += 1
    return word_counts





In [14]:
def word_freq_distribution(word_counts):
    freq_dist = {}
    max_freq = max(word_counts.values())
    for word in word_counts.keys():  
        freq_dist[word] = (word_counts[word]/max_freq)
    return freq_dist


In [15]:
def score_sentences(sents, freq_dist, max_len=40):
    sent_scores = {}  
    for sent in sents:
        words = sent.split(' ')
        for word in words:
            if word.lower() in freq_dist.keys():
                if len(words) < max_len:
                    if sent not in sent_scores.keys():
                        sent_scores[sent] = freq_dist[word.lower()]
                    else:
                        sent_scores[sent] += freq_dist[word.lower()]
    return sent_scores

In [16]:
def summarize(sent_scores, k):
    top_sents = Counter(sent_scores) 
    summary = ''
    scores = []
    
    top = top_sents.most_common(k)
    for t in top: 
        summary += t[0].strip()+'. '
        scores.append((t[1], t[0]))
    return summary[:-1], scores

In [17]:
def function(url_content):
  tokens = tokenizer(url_content)
  sents = sent_tokenizer(url_content)
  word_counts = count_words(tokens)
  freq_dist = word_freq_distribution(word_counts)
  sent_scores = score_sentences(sents, freq_dist)
  summary, summary_sent_scores = summarize(sent_scores, 3)
  return summary


In [32]:

app = Flask(__name__)
run_with_ngrok(app)

@app.route("/" , methods=["GET" , "POST"])
def index():
	if request.method == "POST":
		url = request.form.get("url")
		url_content = get_content(url)
		summary = function(url_content)
		return summary
	return render_template("Page-1.html")


if __name__ == "__main__":
  print("Start running server")
  app.run()


  



Start running server
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://f3d629f8bb7d.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


In [ ]:
soup = BeautifulSoup('https://fr.hespress.com/161720-ligue-europa-bounou-et-en-nesyri-sacres-avec-seville-face-a-linter.html')

/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:336: UserWarning: "https://fr.hespress.com/161720-ligue-europa-bounou-et-en-nesyri-sacres-avec-seville-face-a-linter.html" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


In [ ]:
  id.join([n for n in url[24:31] if n.isdigit()])

'161720'